<a href="https://colab.research.google.com/github/frecklehead/ML-from-scratch/blob/master/Rnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np

class SimpleRNN:
    def __init__(self, input_size, hidden_size, output_size):
        self.hidden_size = hidden_size

        self.U = np.random.randn(hidden_size, input_size) * 0.01
        self.W = np.random.randn(hidden_size, hidden_size) * 0.01
        self.V = np.random.randn(output_size, hidden_size) * 0.01

        self.bh = np.zeros((hidden_size, 1))
        self.bo = np.zeros((output_size, 1))

    def forward(self, inputs):
        h = np.zeros((self.hidden_size, 1))
        self.last_inputs = inputs
        self.last_hs = {0: h}

        for i, x in enumerate(inputs):
            h = np.tanh(self.U @ x + self.W @ h + self.bh)
            self.last_hs[i + 1] = h

        o = self.V @ h + self.bo
        return o

    def backward(self, d_L_d_y, learn_rate=0.001):
        n = len(self.last_inputs)
        d_V = d_L_d_y @ self.last_hs[n].T
        d_bo = d_L_d_y

        d_h = self.V.T @ d_L_d_y
        d_U = np.zeros_like(self.U)
        d_W = np.zeros_like(self.W)
        d_bh = np.zeros_like(self.bh)

        for t in reversed(range(n)):
            temp = ((1 - self.last_hs[t + 1] ** 2) * d_h)
            d_bh += temp
            d_U += temp @ self.last_inputs[t].T
            d_W += temp @ self.last_hs[t].T
            d_h = self.W.T @ temp

        self.U -= learn_rate * d_U
        self.W -= learn_rate * d_W
        self.V -= learn_rate * d_V
        self.bh -= learn_rate * d_bh
        self.bo -= learn_rate * d_bo

def loss_fn(y_pred, y_true):
    return np.mean((y_pred - y_true) ** 2)

def d_loss_fn(y_pred, y_true):
    return 2 * (y_pred - y_true) / y_true.size

# Example usage
input_size = 2
hidden_size = 3
output_size = 1

rnn = SimpleRNN(input_size, hidden_size, output_size)
inputs = [np.random.randn(input_size, 1) for _ in range(5)]
target = np.random.randn(output_size, 1)

output = rnn.forward(inputs)
loss = loss_fn(output, target)
d_L_d_y = d_loss_fn(output, target)
rnn.backward(d_L_d_y)
